## Let's download the necessary Libraries

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
import json # This Library handles Json FIles
from geopy.geocoders import Nominatim #it converts an address into latitude and Longitude

import matplotlib.cm as cm
import matplotlib.colors as colors
# Now we'll import K-Means from cluster stage
from sklearn.cluster import KMeans

#Librasry to handle requests
import requests
# to transform JSON files to Pandas Dataframe
from pandas.io.json import json_normalize

#map rendering Library
import folium
print ('Libraries Imported!!')

Libraries Imported!!


## Downloading and Exploring the dataset

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

In [4]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


### Now we'll load the dataset

In [5]:
with open('newyork_data.json') as json_data:
    newyork_data=json.load(json_data)

Let's take a look at the dataset

In [6]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

## the relevant data lies in the Features Section, We will have to somehow store this data

In [8]:
neighborhoods_data=newyork_data['features']

In [10]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

### Now We'll transform the dataset into pandas DataFrame

In [11]:
column_names =['Borough' , 'Neighborhood','Latitude','Longitude']
neighborhoods=pd.DataFrame(columns=column_names)

In [12]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Now start filling the rows one by one

In [15]:
for data in neighborhoods_data:
    borough=data['properties']['borough']
    neighborhood_name=data['properties']['name']
    
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    #append data in the rows
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [16]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Just to make sure that the dataframe has 5 boroughs and 306 neighborhoods

In [18]:
print("The Dataframe has {} boroughs and {} neighborhoods".format(len(neighborhoods['Borough'].unique()),neighborhoods.shape[0]))

The Dataframe has 5 boroughs and 306 neighborhoods


### Before Creating Manhattan's map , first lets visualize the map of New York City

we'll use geopy library to get the latitude and Longitude of the New York City

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>

In [21]:
address='New York city, NY'
geolocator=Nominatim(user_agent='ny_explorer')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print("The geographical location of New York City is {},{}".format(latitude,longitude))

The geographical location of New York City is 40.7127281,-74.0060152


### Now we'll create the map of New York City with it's Neighborhoods superimposed on top using the folium Library

In [43]:
map_newyork=folium.Map(location=[latitude,longitude],zoom_start=10)
# Adding Markers in the Map
for lat,lon,borough,neighborhood in zip(neighborhoods['Latitude'],neighborhoods['Longitude'],neighborhoods['Borough'],neighborhoods['Neighborhood']):
    label='{},{}'.format(neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lon],radius=5,color='#560A86',fill=True,fillcolor='#7122FA',fill_opacity=0.6,parse_html=False).add_to(map_newyork)

In [44]:
map_newyork

### Now we'll simplify the above map and segment and cluster the neighborhoods in Manhattan

We'll create a new dataframe named manhattan_data

In [45]:
manhattan_data=neighborhoods[neighborhoods['Borough']=='Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Now, let's get the geograophical coordinates of manhattan

In [46]:
address='Manhattan,NY'
geolocator=Nominatim(user_agent='ny_explorer')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print("the geographical coordinates of Manhattan are {},{}".format(latitude,longitude))

the geographical coordinates of Manhattan are 40.7896239,-73.9598939


Now , Let's visualize the neighborhoods in Manhattan

In [57]:
map_manhattan=folium.Map(location=[latitude,longitude],zoom_start=11)

#adding markers to the map
for lat,lon ,label in zip(manhattan_data['Latitude'],manhattan_data['Longitude'],manhattan_data['Neighborhood']):
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lon],radius=5,popup=label,color='blue',fill=True ,fill_color='green',fill_opacity=0.7,parse_html=False).add_to(map_manhattan)
    


In [58]:
map_manhattan

## Now, we'll make use of the Foursquare API to  explore the neighborhoods and segment them

In [60]:
CLIENT_ID = 'DF0VYKOXY13ZE1DB0TSQN1PRP4CR0YVRSMQSXYTYPAVQIQ1V' # your Foursquare ID
CLIENT_SECRET = 'YBOCDU3AJ0ZIDJO03TMXR2FWVXUMGJYCNQSJEFMXSD01WR5S' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DF0VYKOXY13ZE1DB0TSQN1PRP4CR0YVRSMQSXYTYPAVQIQ1V
CLIENT_SECRET:YBOCDU3AJ0ZIDJO03TMXR2FWVXUMGJYCNQSJEFMXSD01WR5S


### let's explore the first neighborhood in our dataframe

In [61]:
manhattan_data.loc[0,'Neighborhood']

'Marble Hill'

Get the neighborhood's latitude and longitude values

In [63]:
neighborhood_latitude=manhattan_data.loc[0,'Latitude']
neighborhood_longitude=manhattan_data.loc[0,'Longitude']
neighborhood_name=manhattan_data.loc[0,'Neighborhood']
print("Latitude,Longitude values of {} are {},{}".format(neighborhood_name,neighborhood_latitude,neighborhood_longitude))


Latitude,Longitude values of Marble Hill are 40.87655077879964,-73.91065965862981


## Now let's get the top 100 venues in Marble Hill within a radius of 500 metres

In [73]:
# Limit of number of venues returned by the Foursquare API
LIMIT=100
radius=500 #in metres

#now we need to create a url
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=DF0VYKOXY13ZE1DB0TSQN1PRP4CR0YVRSMQSXYTYPAVQIQ1V&client_secret=YBOCDU3AJ0ZIDJO03TMXR2FWVXUMGJYCNQSJEFMXSD01WR5S&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

In [74]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edc07a97828ae001b1c2568'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 26,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label'

all the information lies in the items key

In [75]:
# We'll define a function that extracts the category of the venue
def get_category_type(row):
    try:
        catagories_list=row['categories']
    except:
        categories_list=row['venue.categories']
        
    if len(categories_list==0):
        return None
    else:
        return categories_list[0]['name']

## Exploring Neighborhoods in Manhattan

Now , we'll clean the json format and structure it into pandas dataframe

In [78]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [80]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )



Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


Let's check the size of the resulting dataframe

In [81]:
print(manhattan_venues.shape)

(3104, 7)


In [83]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


let's see how many kind of venues are there

In [87]:
manhattan_venues.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Accessories Store,1,1,1,1,1,1
Adult Boutique,1,1,1,1,1,1
Afghan Restaurant,1,1,1,1,1,1
African Restaurant,3,3,3,3,3,3
American Restaurant,69,69,69,69,69,69
Antique Shop,1,1,1,1,1,1
Arcade,1,1,1,1,1,1
Arepa Restaurant,2,2,2,2,2,2
Argentinian Restaurant,5,5,5,5,5,5


As we can see this data is not that useful in our case, therefore we'll check how many venues were returned for each neighborhood

In [85]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,65,65,65,65,65,65
Carnegie Hill,84,84,84,84,84,84
Central Harlem,46,46,46,46,46,46
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,42,42,42,42,42,42
East Village,100,100,100,100,100,100


let's see how many unique categories were returned

In [88]:
print('There are {} unique categories'.format(len(manhattan_venues['Venue Category'].unique())))

There are 328 unique categories


# Now we'll analyze each neighborhood

In [90]:
#one hot encoding
manhattan_onehot=pd.get_dummies(manhattan_venues[['Venue Category']],prefix="",prefix_sep="")
#adding neighborhood column to the dataframe
manhattan_onehot['Neighborhood']=manhattan_venues['Neighborhood']
# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,

In [91]:
manhattan_onehot.shape

(3104, 329)

#### Next, we'll group the rows by neighborhood and will take the mean of the frequency of occurrence of each category

In [93]:
manhattan_grouped=manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,

In [95]:
manhattan_grouped.shape # this is the new size

(40, 329)

### Now, let's print the neighborhoods with five of the most common venues

In [98]:
num_top_venues =5
for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    

----Battery Park City----
           venue  freq
0           Park  0.12
1          Hotel  0.06
2  Memorial Site  0.05
3    Coffee Shop  0.05
4            Gym  0.05


----Carnegie Hill----
         venue  freq
0  Coffee Shop  0.10
1         Café  0.05
2  Pizza Place  0.05
3  Yoga Studio  0.04
4          Gym  0.04


----Central Harlem----
                  venue  freq
0    African Restaurant  0.07
1     French Restaurant  0.04
2   American Restaurant  0.04
3  Gym / Fitness Center  0.04
4                   Bar  0.04


----Chelsea----
                 venue  freq
0          Coffee Shop  0.08
1          Art Gallery  0.08
2       Ice Cream Shop  0.04
3                 Café  0.03
4  American Restaurant  0.03


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.07
1                 Bakery  0.05
2           Cocktail Bar  0.04
3            Coffee Shop  0.03
4  Vietnamese Restaurant  0.03


----Civic Center----
          venue  freq
0   Coffee Shop  0.08
1         Hotel 

### Now let's put this data into a dataframe

In [99]:
# Function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Now let's create a dataframe that contains the top 5 venues 

In [100]:
num_top_venues=5
indicators=['st','nd','rt']
# create columns according to the number of top venues
columns=['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [101]:
# creating a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rt Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Battery Park City,Park,Hotel,Gym,Coffee Shop,Memorial Site
1,Carnegie Hill,Coffee Shop,Pizza Place,Café,Bookstore,Gym / Fitness Center
2,Central Harlem,African Restaurant,Chinese Restaurant,Gym / Fitness Center,American Restaurant,Cosmetics Shop
3,Chelsea,Art Gallery,Coffee Shop,Ice Cream Shop,Café,American Restaurant
4,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Bubble Tea Shop,Ice Cream Shop


## Now we will Cluster the neighborhoods

### We'll cluster the neighborhood into 5 portions and for this we will use K-Means algorithm 

In [103]:
#set number of clusters
kclusters=5
manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5] 

array([2, 1, 0, 1, 1])

### Let's create a dataframe with top 5 venues as well as the cluster label

In [104]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rt Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Diner
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Bakery,Cocktail Bar,Bubble Tea Shop,Ice Cream Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Café,Bakery,Mobile Phone Shop,Donut Shop,Bank
3,Manhattan,Inwood,40.867684,-73.921210,2,Lounge,Mexican Restaurant,Café,Restaurant,Bakery
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Pizza Place,Coffee Shop,Café,Deli / Bodega,Mexican Restaurant


## Let's plot the clusters on the map of Manhattan using Folium Library

In [105]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Now Let's Examine each cluster and based upon this examination we will suggest a favourable place to open which kind of restaurant

## Cluster 1

In [107]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rt Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,Central Harlem,African Restaurant,Chinese Restaurant,Gym / Fitness Center,American Restaurant,Cosmetics Shop
13,Lincoln Square,Plaza,Café,Italian Restaurant,Performing Arts Venue,Concert Hall
14,Clinton,Theater,Gym / Fitness Center,Italian Restaurant,Coffee Shop,Wine Shop
21,Tribeca,Italian Restaurant,American Restaurant,Park,Wine Bar,Spa
39,Hudson Yards,Italian Restaurant,Hotel,Gym / Fitness Center,Bar,Gym


# Report of cluster 1
We can clearly see that this cluster, which is represented by red color on the map has a lot of Italian Restaurants.
Therefore opening an italian restaurant would be a really bad idea in this region.
In case you just want to open a cafe or a coffee shop this could be a good place since it doesn't have much competition in this sector.

## Cluster 2

In [108]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rt Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Bubble Tea Shop,Ice Cream Shop
8,Upper East Side,Italian Restaurant,Coffee Shop,Gym / Fitness Center,Bakery,Exhibit
9,Yorkville,Coffee Shop,Gym,Italian Restaurant,Bar,Deli / Bodega
10,Lenox Hill,Italian Restaurant,Coffee Shop,Pizza Place,Café,Cocktail Bar
12,Upper West Side,Italian Restaurant,Bar,Wine Bar,Indian Restaurant,Dessert Shop
15,Midtown,Coffee Shop,Hotel,Bakery,Theater,Clothing Store
16,Murray Hill,Hotel,Sandwich Place,Pizza Place,Gym / Fitness Center,Japanese Restaurant
17,Chelsea,Art Gallery,Coffee Shop,Ice Cream Shop,Café,American Restaurant
18,Greenwich Village,Italian Restaurant,Café,Sushi Restaurant,Gym,Chinese Restaurant
19,East Village,Bar,Mexican Restaurant,Coffee Shop,Cocktail Bar,Japanese Restaurant


## Report of Cluster 2 

This cluster covers the majority of the region in Manhattan ,and it's clearly visible that it has a lot of restaurants and in such huge amounts ,so this region should not be considered for opening any restaurant or a cafe.

## Cluster 3

In [109]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rt Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Donut Shop,Bank
3,Inwood,Lounge,Mexican Restaurant,Café,Restaurant,Bakery
4,Hamilton Heights,Pizza Place,Coffee Shop,Café,Deli / Bodega,Mexican Restaurant
5,Manhattanville,Coffee Shop,Seafood Restaurant,Mexican Restaurant,Park,Italian Restaurant
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Latin American Restaurant,Deli / Bodega
11,Roosevelt Island,Bubble Tea Shop,Residential Building (Apartment / Condo),Sandwich Place,Liquor Store,Dry Cleaner
26,Morningside Heights,Park,Coffee Shop,Bookstore,American Restaurant,Burger Joint
28,Battery Park City,Park,Hotel,Gym,Coffee Shop,Memorial Site
36,Tudor City,Café,Park,Mexican Restaurant,Pizza Place,Sushi Restaurant


## Report of cluster 3

This region does't have a lot of restaurants especially if someone wants to open an italian restaurant....this is the most favourable region to do so.
Especially, near the Colombia University in the 116th street. It doesn't have much competition and this region is well populated as well. 

## Cluster 4

In [112]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rt Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,Stuyvesant Town,Boat or Ferry,Playground,Park,Bar,Farmers Market


## Report of cluster 4

So, this place may not be good for opening any restaurant because this region basically consists of Boat Yards and people who would come in this region are most likely to go and enjoy their time on the boat.
However ,if someone wants to open a takeaway restaurant it might be a good place because people who are going on boat may like to carry some food with them.

## Cluster 5

In [113]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rt Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Marble Hill,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Diner


## Report of cluster 5

This region is slightly away from the residential area ,so there might not be that many customers present in this region. Therefore, I would not recomment opening a restaurant here.

# Final Recommendation by me

If someone actually wants to open a restaurant in Manhattan and wants to make profit out of it , they should open an italian restaurant and that to in the regions of cluster 3.    
    
Why Italian?
Because, the data above directs us to a conclusion that people in New York City love to eat Italian Cuisine.
    
What would be the best spot?
According to me , the best spot would be somewhere near 116th street near Columbia University.